In [276]:
# Calculate Distance - Notebook
# Author: S. Chu
# Date: 2024-10-27

# Setup:
from geopy.geocoders import Nominatim
from geopy.distance import great_circle
import folium

geolocator = Nominatim(user_agent="Hello World")

# Define set of cities:
cities = [
    "Asheville, NC",
    "San Diego, CA",
    "Pearl Harbor, HI",
    "Munich, Germany",
    "Yokohama, Japan",
    "London, England",
    "Groton, CT"
]

# Target location (Coordinate):
target = "Washington, DC"
target1 = (38.895037, -77.036543)

# Functions
# Calculate distance (city, distance from target):
def calculate_distances(cities, target_location):
    distances = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (location.latitude, location.longitude)
            distance = great_circle(city_coordinate, target_location).kilometers
            distances[city] = round(distance, 1)
        else:
            distances[city] = None
    return distances

# Get coordinates (city, coordinates):
def get_coordinates(cities, target_location):
    coordinates = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (
                round(location.latitude, 6),
                round(location.longitude, 6)
            )
            coordinates[city] = city_coordinate
        else:
            coordinates[city] = None
    return coordinates

# Get data and sort (city, coordinates, distance from target):
def sorted_distances(cities, target_location):
    distances = {}
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (
                round(location.latitude, 6),
                round(location.longitude, 6)
            )
            distance = great_circle(city_coordinate, target_location).kilometers
            distances[city] = (city, city_coordinate, round(distance, 1))
        else:
            distances[city] = None
            distances[city] = (city, 'N/A', 'N/A')
    sorted_distances = sorted(
        distances.values(), key=lambda x: x[2]
        if x[2] != 'N/A' else float('inf')      # Substitutes 'N/A' as 'inf'
        )
    print_data(sorted_distances)
    return sorted_distances

# Print sorted distances - Used with sorted_distances:
def print_data(sorted_data):
    print("\nSorted Distances:")
    print(f"{'City':<25} {'Coordinates':<30} {'Distance (km)':<15}")  # Header
    for city, coord, dist in sorted_data:
        if dist != 'N/A':
            formatted_coordinates = f"({coord[0]}, {coord[1]})"
            print(f"{city:<25} {formatted_coordinates:<30} {dist:>10.1f} km")
        else:
            print(f"{city:<25} {'N/A':<30} {'N/A':>10}")
    return sorted_data

# Target Location or Coordinate:
def is_coordinate(location):
    if isinstance(location, tuple) and len(location) == 2:
        latitude, longitude = location
        if isinstance(latitude, (int,float)) and isinstance(longitude, (int,float)):
            if -90 <= latitude <= 90 and -180 <= longitude <= 180:
                return True
    return False

# Reverse coordinate search:
def reverse_coordinate_search(target_location):
    location = geolocator.reverse(target_location)
    return location

# Find location coordinate:
def get_coordinate(target_location):
    location = geolocator.geocode(target_location)
    coordinate = (
        round(location.latitude, 6),
        round(location.longitude, 6)
        )
    return coordinate

# Get target name and coordinates
def check_target(target_location):
    if is_coordinate(target_location) == False:
        target_name = target_location
        target_coordinates = get_coordinate(target_location)
    else:
        target_name = reverse_coordinate_search(target_location)
        target_coordinates = target_location
    return target_name, target_coordinates

# Function to calculate midpoint between two coordinates
def midpoint(coord1, coord2):
    return [(coord1[0] + coord2[0]) / 2, (coord1[1] + coord2[1]) / 2]


In [277]:
# Call function to determine target coordinate
# Choose between target (name) vs target1 (coordinate)
target_name, target_coordinate = check_target(target)
print(f"{'Target Name'}: {target_name} - {'Target Coordinates'}: {target_coordinate}")


Target Name: Washington, DC - Target Coordinates: (38.895037, -77.036543)


In [278]:
# Call functions
distances = calculate_distances(cities, target_coordinate)
coordinates = get_coordinates(cities, target_coordinate)
output = sorted_distances(cities, target_coordinate)



Sorted Distances:
City                      Coordinates                    Distance (km)  
Groton, CT                (41.35016, -72.0762)                502.3 km
Asheville, NC             (35.595363, -82.550841)             610.4 km
San Diego, CA             (32.71742, -117.162772)            3653.1 km
London, England           (51.507446, -0.127765)             5898.5 km
Munich, Germany           (48.137108, 11.575382)             6816.4 km
Pearl Harbor, HI          (21.355775, -157.999149)           7780.3 km
Yokohama, Japan           (35.444395, 139.636773)           10930.9 km


In [279]:
# Create map with distances:
def map_distances(cities, target_location, target_name):
    distances = {}

    # Initalize map centered on target location
    m = folium.Map(location=target_location, zoom_start=5)
    folium.Marker(
        location = target_location,
        popup = f"{target_name}: {target_location}",
        icon = folium.Icon(color="red")
    ).add_to(m)

    # Calculate distances and mark on map
    for city in cities:
        location = geolocator.geocode(city)
        if location:
            city_coordinate = (location.latitude, location.longitude)
            distance = great_circle(city_coordinate, target_location).kilometers
            distances[city] = (city_coordinate, round(distance, 1))

            # Add city marker on map
            folium.Marker(
                location = city_coordinate,
                popup = f"{city}: {distances[city][1]} km",
                icon = folium.Icon(color="blue")
            ).add_to(m)

            # Draw line between target location and city on map
            folium.PolyLine(
                [target_location, city_coordinate],
                color = "blue",
                weight = 2,
                opacity = 0.5
            ).add_to(m)

            ## Add a distance marker at the midpoint
            mid_point = midpoint(target_location, city_coordinate)
            folium.Marker(
                location=mid_point,
                icon=folium.DivIcon(html=f"""<div style="font-size: 10pt; color : black">{distances[city][1]} km
                                    </div>""")
            ).add_to(m)
        else:
            distances[city] = ('N/A', 'N/A')

    return m, distances


In [280]:
# Generate the map
map_obj, distances = map_distances(cities, target_coordinate, target_name)

# Display the map
map_obj
